## GenerativeAI4DS-II
### Healthcare Lab (Sequential Chains)


### In this lab you will develop sequential chains using LangChain.
### Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step.

### Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [3]:
#OPENAI_API_KEY="YOU-NEED-YOUR-OWN-KEY"

In [4]:
!pip install --upgrade langchain;

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 604.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 12.7 MB/s eta 0:00:00


In [5]:

!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00


In [6]:
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.6 MB/s eta 0:00:00


In [7]:
import os
import openai



In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [9]:
from langchain.chains import SequentialChain

### 1. First LLMChain


In [31]:
llm1 = ChatOpenAI(temperature=0.0, model='gpt-3.5-turbo',openai_api_key=OPENAI_API_KEY)

first_prompt = ChatPromptTemplate.from_template(
    "Produce a JSON file from the following medical information:"
    "\n\n{Patient_Information}"
)

In [32]:
# chain 1 input= Patient's medical information
chain_one = LLMChain(llm=llm1, prompt=first_prompt,
                     output_key="Patient_Information_JSON"
                    )

### 2. Second LLMChain

In [33]:
llm2 = ChatOpenAI(temperature=0.0, model='gpt-3.5-turbo',openai_api_key=OPENAI_API_KEY)

second_prompt = ChatPromptTemplate.from_template(
    "Provide a medical diagnosis given the following medical information:"
    "\n\n{Patient_Information}"
)

In [34]:
# chain 2 input= Patient's medical information
chain_two = LLMChain(llm=llm2, prompt=second_prompt,
                     output_key="Medical_Diagnosis"
                    )

### 3. Third LLMChain

In [35]:
llm3 = ChatOpenAI(temperature=0.0, model='gpt-3.5-turbo',openai_api_key=OPENAI_API_KEY)

third_prompt = ChatPromptTemplate.from_template(
    "Provide a medical prognosis given the following medical information and medical diagnosis"
    "\n\n{Patient_Information}{Medical_Diagnosis}"
)

In [36]:
# chain 3 input= Patient's medical information
chain_three= LLMChain(llm=llm3, prompt=third_prompt,
                     output_key="Medical_Prognosis"
                    )

### 5. Composite Chain
#### We combine the previous chains into a composite one

In [38]:
# composite_chain: input= Physical_Examination
# and output= Medical_Diagnosis

composite_chain = SequentialChain(
    chains=[chain_one,chain_two,chain_three],
    input_variables=["Patient_Information"],
    output_variables=["Patient_Information_JSON","Medical_Diagnosis","Medical_Prognosis"],
    verbose=True)

### 5. Chain Execution

In [39]:
Mark_Thompson_Medical_Information='''

**Patient Information:**
- **Name:** John Doe
- **Age:** 62 years old
- **Gender:** Male
- **Medical History:** Type 2 diabetes mellitus, hypertension, chronic pancreatitis
- **Family History:** Father diagnosed with colon cancer at age 70

**Presenting Symptoms:**
- Unintentional weight loss of 15 pounds over 3 months
- Persistent upper abdominal pain radiating to the back
- Jaundice (yellowing of the skin and eyes)
- Dark urine and pale stools
- Loss of appetite and early satiety

**Physical Examination:**
- Cachexia (severe weight loss)
- Icterus (jaundice)
- Palpable non-tender gallbladder (Courvoisier's sign)
- Epigastric tenderness

**Diagnostic Workup:**

1. **Laboratory Tests:**
   - Complete blood count (CBC): Normocytic anemia
   - Liver function tests (LFTs): Elevated bilirubin, alkaline phosphatase, and gamma-glutamyl transferase (GGT)
   - Tumor markers: Elevated CA 19-9 and CEA levels

2. **Imaging Studies:**
   - **Abdominal Ultrasound:**
     - Dilated bile ducts
     - Mass in the head of the pancreas
   - **Contrast-Enhanced CT Scan of the Abdomen:**
     - Hypoattenuating mass in the head of the pancreas measuring 4.5 cm
     - Dilatation of the pancreatic and bile ducts (double duct sign)
     - Involvement of the superior mesenteric artery
     - Suspicious lesions in the liver suggestive of metastasis
   - **MRI/MRCP:**
     - Detailed view of the biliary tree and pancreatic ducts confirming CT findings

3. **Biopsy:**
   - **Endoscopic Ultrasound-Guided Fine Needle Aspiration (EUS-FNA):**
     - Cytology: Adenocarcinoma cells


'''

In [46]:
composite_chain_output=composite_chain(Mark_Thompson_Medical_Information)



> Entering new SequentialChain chain...

> Finished chain.


In [47]:
type(composite_chain_output)

dict

In [48]:
composite_chain_output['Patient_Information_JSON']

'{\n    "Patient Information": {\n        "Name": "John Doe",\n        "Age": "62 years old",\n        "Gender": "Male",\n        "Medical History": ["Type 2 diabetes mellitus", "Hypertension", "Chronic pancreatitis"],\n        "Family History": "Father diagnosed with colon cancer at age 70"\n    },\n    "Presenting Symptoms": [\n        "Unintentional weight loss of 15 pounds over 3 months",\n        "Persistent upper abdominal pain radiating to the back",\n        "Jaundice (yellowing of the skin and eyes)",\n        "Dark urine and pale stools",\n        "Loss of appetite and early satiety"\n    ],\n    "Physical Examination": [\n        "Cachexia (severe weight loss)",\n        "Icterus (jaundice)",\n        "Palpable non-tender gallbladder (Courvoisier\'s sign)",\n        "Epigastric tenderness"\n    ],\n    "Diagnostic Workup": {\n        "Laboratory Tests": {\n            "Complete blood count (CBC)": "Normocytic anemia",\n            "Liver function tests (LFTs)": "Elevated bil

In [49]:
composite_chain_output['Medical_Diagnosis']

"Based on the patient's medical history, presenting symptoms, physical examination findings, and diagnostic workup results, the medical diagnosis for John Doe is **Pancreatic Adenocarcinoma**. \n\nPancreatic adenocarcinoma is a type of cancer that originates in the cells lining the ducts of the pancreas. The presence of a mass in the head of the pancreas, dilatation of the bile ducts, elevated tumor markers (CA 19-9 and CEA), and confirmation of adenocarcinoma cells on biopsy all point towards this diagnosis. The involvement of the superior mesenteric artery and suspicious lesions in the liver suggest advanced disease with possible metastasis. \n\nTreatment options for pancreatic adenocarcinoma may include surgery, chemotherapy, radiation therapy, and palliative care to manage symptoms and improve quality of life. It is important for John Doe to discuss treatment options with his healthcare team to determine the best course of action for his specific case."

In [50]:
composite_chain_output['Medical_Prognosis']

"The prognosis for pancreatic adenocarcinoma is generally poor, as it is often diagnosed at an advanced stage when the cancer has already spread. The 5-year survival rate for pancreatic cancer is low, around 10%. However, survival rates can vary depending on the stage of the cancer at diagnosis, the overall health of the patient, and the effectiveness of treatment.\n\nGiven John Doe's advanced presentation with involvement of surrounding structures and possible liver metastasis, his prognosis may be guarded. It is important for him to work closely with his healthcare team to develop a comprehensive treatment plan and to address any symptoms or complications that may arise. Palliative care may also play a role in managing pain and improving quality of life.\n\nRegular follow-up appointments and monitoring will be essential to track the progression of the cancer and response to treatment. It is important for John Doe to maintain open communication with his healthcare providers and to see